# Setup

In [1]:
import numpy as np
import math
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup

In [2]:
np.random.seed(999)

In [3]:
REVIEWS_PATH  = "./data/Video_Games_5.json.gz"
METADATA_PATH = "./data/meta_Video_Games.json.gz"

In [4]:
REVIEWS_FEATS = ['asin', 'reviewerID', 'overall']
METADATA_FEATS = ['asin', 'title', 'category']

In [5]:
NUM_PRODUCTS = None
K = 10

# Load

## Reviews

In [6]:
reviews = pd.read_json(REVIEWS_PATH, lines=True, compression='gzip', encoding = 'utf-8')
reviews = reviews[REVIEWS_FEATS]
reviews.head(5)

,asin,reviewerID,overall
0,0700026657,A1HP7NVNPFMA4N,5
1,0700026657,A1JGAP0185YJI6,4
2,0700026657,A1YJWEXHQBWK2B,3
3,0700026657,A2204E1TH211HT,2
4,0700026657,A2RF5B5H74JLPE,5


## Metadata

In [7]:
metadata = pd.read_json(METADATA_PATH, lines=True, compression='gzip', encoding = 'utf-8')
metadata = metadata[METADATA_FEATS]
metadata.head(5)

,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[Video Games, PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Video Games, Xbox 360, Games, </span></span><..."
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Video Games, Retro Gaming & Microconsoles, Su..."
3,0324411812,Xbox 360 MAS STICK,"[Video Games, Xbox 360, Accessories, Controlle..."
4,0439335310,Phonics Alive! 3: The Speller,"[Video Games, PC, Games, </span></span></span>..."


### Clean category data

In [8]:
clean_HTML     = lambda vals: [BeautifulSoup(val, 'html.parser').get_text() for val in vals]
clean_empties  = lambda vals: [val for val in vals if val]

clean_pipeline = lambda vals: clean_empties(clean_HTML(vals))

In [9]:
metadata['category'] = metadata['category'].apply(clean_pipeline)
metadata.head(5)

/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[Video Games, PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Video Games, Xbox 360, Games]"
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Video Games, Retro Gaming & Microconsoles, Su..."
3,0324411812,Xbox 360 MAS STICK,"[Video Games, Xbox 360, Accessories, Controlle..."
4,0439335310,Phonics Alive! 3: The Speller,"[Video Games, PC, Games, Grades 2-12, Spelling..."


### Select category features

In [10]:
# Explode the 'cleaned_categories' column to create a row for each category
# Use value_counts to count the occurrences of each category
unique_cats = metadata['category'].explode().reset_index(drop=True).value_counts().reset_index()
unique_cats.columns = ['category', 'frequency']

In [11]:
select_cats = unique_cats[
                (unique_cats.frequency > 25) &
                # filter long descriptions
                (unique_cats.category.str.len() < 40) &
                # filter stray characters
                (unique_cats.category.str.len() > 1)
            ]['category']
select_cats = set(select_cats.values)
select_cats

{'1 Player!',
 '10 Different levels',
 '10 Fun Levels',
 '100% satisfaction guaranteed',
 '160 Objects to Find',
 '3DO',
 '4 Different levels',
 '40 Objects Per Level',
 '400 Hidden Objects to Find',
 '400 Objects to Find',
 'Accessories',
 'Accessory Kits',
 'Adapters',
 'Anything else is just a sticker!!',
 'Atari 2600',
 'Atari 7800',
 'Atari Jaguar',
 'Atari Lynx',
 'Batteries',
 'Batteries & Chargers',
 'Brand new and high quality',
 'Cables',
 'Cables & Adapters',
 'Cartridge only.',
 'Cases & Storage',
 'Chargers',
 'Commodore 64',
 'Complete housing replacement set.',
 'Consoles',
 'Controllers',
 'Cooling Systems',
 'Currency & Subscription Cards',
 'Currency Cards',
 'Dance Pads',
 'Decorate and beautify your console',
 'Designed and sold by Demon Decal',
 'Digital Games',
 'Digital Games & DLC',
 'Downloadable Content',
 'Drums',
 'Easy to apply, clean, and remove',
 'Easy to use, stick, clean and remove',
 'Faceplates',
 'Faceplates, Protectors & Skins',
 'Features -',
 'Fi

In [12]:
select_pipeline = lambda vals: [val for val in vals if val in select_cats]

In [13]:
metadata['category'] = metadata['category'].apply(select_pipeline)
metadata.head(5)

,asin,title,category
0,0042000742,Reversi Sensory Challenger,"[Video Games, PC, Games]"
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,"[Video Games, Xbox 360, Games]"
2,0276425316,street fighter 2 II turbo super nintendo snes ...,"[Video Games, Retro Gaming & Microconsoles, Su..."
3,0324411812,Xbox 360 MAS STICK,"[Video Games, Xbox 360, Accessories, Controlle..."
4,0439335310,Phonics Alive! 3: The Speller,"[Video Games, PC, Games]"


## Join datasets

In [14]:
data = reviews.merge(metadata, how='inner', on='asin')
data

,asin,reviewerID,overall,title,category
0,0700026657,A1HP7NVNPFMA4N,5,Anno 2070,"[Video Games, PC, Games]"
1,0700026657,A1JGAP0185YJI6,4,Anno 2070,"[Video Games, PC, Games]"
2,0700026657,A1YJWEXHQBWK2B,3,Anno 2070,"[Video Games, PC, Games]"
3,0700026657,A2204E1TH211HT,2,Anno 2070,"[Video Games, PC, Games]"
4,0700026657,A2RF5B5H74JLPE,5,Anno 2070,"[Video Games, PC, Games]"
...,...,...,...,...,...
568981,B01H7VI5TC,A2Q5FXGX0VOWNV,4,Two Pack N64 Nintendo 64 Extension Cables,"[Video Games, Retro Gaming & Microconsoles, Ni..."
568982,B01H7VI5TC,A2972RZ8R4SBSZ,5,Two Pack N64 Nintendo 64 Extension Cables,"[Video Games, Retro Gaming & Microconsoles, Ni..."
568983,B01H7VI5TC,A1NBY361391RVJ,5,Two Pack N64 Nintendo 64 Extension Cables,"[Video Games, Retro Gaming & Microconsoles, Ni..."
568984,B01H7VI5TC,A2TIZCOP1KN2YA,5,Two Pack N64 Nintendo 64 Extension Cables,"[Video Games, Retro Gaming & Microconsoles, Ni..."


### Subset

In [15]:
# Subset the data
if NUM_PRODUCTS:
    sample_asins = np.random.choice(data['asin'].unique(), size=NUM_PRODUCTS, replace=False)
    data = data[data['asin'].isin(sample_asins)]
data.shape

(568986, 5)

## Helpers

In [16]:
def get_metadata(asin):
    return metadata[metadata['asin'] == asin].iloc[0]

## Compute features

### User rating features

In [17]:
user_item_matrix = data.pivot_table(index='reviewerID', columns='asin', values='overall').fillna(0)
item_user_matrix = user_item_matrix.T
item_user_matrix.shape

(17389, 55223)

In [18]:
rating_similarity = cosine_similarity(item_user_matrix)
rating_similarity.shape

(17389, 17389)

### Category features

In [19]:
categories = data[['asin', 'category']]
categories = categories.set_index('asin')
categories = categories[~categories.index.duplicated()]

# concatenate the categories into single strings
SEP = ";"
categories['category_str'] = categories['category'].apply(lambda x: SEP.join(x))

print(categories.shape)
categories.head(5)

(17389, 2)


,category,category_str
asin,,
0700026657,"[Video Games, PC, Games]",Video Games;PC;Games
0700099867,"[Video Games, PC, Games]",Video Games;PC;Games
0700026398,"[Video Games, PC, Games]",Video Games;PC;Games
0804161380,[],
3828770193,"[Video Games, Kids & Family, Nintendo DS, Games]",Video Games;Kids & Family;Nintendo DS;Games


In [20]:
# TF-IDF using a custom tokenizer
tokenizer = lambda s: [val for val in s.split(SEP) if val]
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=True)
tfidf_matrix = vectorizer.fit_transform(categories['category_str'])
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), index=categories.index, columns=vectorizer.get_feature_names_out())
tfidf_matrix.head(5)

,1 player!,10 different levels,10 fun levels,3do,40 objects per level,400 hidden objects to find,400 objects to find,accessories,accessory kits,adapters,...,third party product,thumb grips,turbografx 16,video games,wii,wii u,"will not scratch, fade or peel",xbox,xbox 360,xbox one
asin,,,,,,,,,,,,,,,,,,,,,
0700026657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.335101,0.0,0.0,0.0,0.0,0.0,0.0
0700099867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.335101,0.0,0.0,0.0,0.0,0.0,0.0
0700026398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.335101,0.0,0.0,0.0,0.0,0.0,0.0
0804161380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3828770193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.197580,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
cat_similarity = linear_kernel(tfidf_matrix)
cat_similarity.shape

(17389, 17389)

### Combine into one similarity matrix

In [22]:
def weighted_sim(feats: tuple, weights: tuple = None):
    '''
    feats: tuple of feature vectors
    weights: optional tuple of corresponding weights
             if None or 'equal', then use equal weights
             if 'scale', then scale by mean values
    '''    
    if not weights or weights == 'equal':
        denom = len(feats)
        weights = tuple([(1/denom) for _ in feats])
    elif weights == 'scale':
        denom = sum([feat.mean() for feat in feats])
        weights = tuple([(1 - feat.mean() / denom) for feat in feats])
        
    similarity_matrix = np.sum([ coefs[0]*coefs[1] for coefs in zip(weights, feats) ], axis=0)
    similarity_matrix = pd.DataFrame(similarity_matrix, index=user_item_matrix.columns, columns=user_item_matrix.columns)
    
    # cosine similarity is in range (0, 1), so compute distance as 1 - similarity
    # use clip to handle floating point precision errors
    dist_matrix = np.clip((1 - similarity_matrix.values), a_min=0, a_max=None)
    dist_matrix = pd.DataFrame(dist_matrix, index=similarity_matrix.index, columns=similarity_matrix.columns)
    
    return similarity_matrix, dist_matrix

# Item-based collaborative filtering - Using k-NN
Similarity and ranking using learned k-NN model

## Implementation

In [23]:
def recall(actual, predicted):
    actual = set(actual)
    predicted = set(predicted)
    hits = len(predicted & actual)
    recall = hits / len(actual)
    return recall

def precision(actual, predicted, k=None):
    if not k:
        k = len(predicted)
    actual    = set(actual)
    predicted = set(predicted[:k])
    hits = len(predicted & actual)
    precision = hits / k if k > 0 else 0
    return precision

def evaluate_recs(target_items, recommended_items, k=K):
    '''
    Soft evaluations using reviews as a signal for successful recommendations
    Returns: hits, recall, precision, precision@k
    '''
    hits = len( set(target_items) & set(recommended_items) )
    r    = recall(target_items, recommended_items)
    p    = precision(target_items, recommended_items)
    p_k  = precision(target_items, recommended_items, k=k)
    return (hits, r, p, p_k,)

In [24]:
def query(query_user, model, k=K, show_eval=False):
    '''
    Query user is a user/reviewerID to create recommendations for.
    Model is a k-NN model.
    '''
    print(f'Querying user: {query_user}')

    # Choose positive reviews as query items
    print('\nPositive reviews: ')
    query_user_reviews = user_item_matrix.loc[query_user].T
    query_user_reviews = query_user_reviews.loc[query_user_reviews > 3]
    print(query_user_reviews)

    if not query_user_reviews.any():
        print('No positive reviews to reference.')
    else:
        # store the results
        recommendations = dict()
        recommendations_attr = dict()

        print('\nReference titles:')
        for query_asin in query_user_reviews.index:
            print(f'  - {get_metadata(query_asin).title} (item {query_asin})')

        # perform the query
        query_matrix = dist_matrix.loc[query_user_reviews.index]
        res_dists, res_idx = model.kneighbors(query_matrix.values, n_neighbors=k)  # rows=queries, cols=k-NN
        res_dists = res_dists.flatten()
        res_idx = res_idx.flatten()

        # collect and attribute
        for i in range(len(res_idx)):
            item = item_user_matrix.iloc[res_idx[i]].name
            item_attr = query_user_reviews.index[i // K]
            # avoid recommending itself
            if item != item_attr:
                recommendations[item] = res_dists[i]
                recommendations_attr[item] = item_attr

        # perform evaluation
        recommendations_sorted = sorted(recommendations, key=recommendations.get)
        hits, r, p, p_k = evaluate_recs(list(query_user_reviews.index), list(recommendations.keys()), k=k)
        
        # remove query overlap
        for item in list(recommendations.keys()):
            if item in query_user_reviews.index:
                recommendations.pop(item)
                recommendations_attr.pop(item)
        
        # select final recommendations to serve
        recommendations_sorted = sorted(recommendations, key=recommendations.get)[:k]
        print(f'\nTop {k} recommendations')
        print('-'*100)
        for i, rec in enumerate(recommendations_sorted):
            query_asin = recommendations_attr[rec]
            print(f"  {i+1}) {get_metadata(rec).title} (item {rec})")
            print(f"    -- Based on {get_metadata(query_asin).title} (item {query_asin})")
            print(f"       Similarity: {similarity_matrix[query_asin].loc[rec]}")
        print('-'*100)
        
    
    if show_eval:
        print('Recommendation evaluation:')
        print(f'\tHits: {hits}')
        print(f'\tRecall: {r}')
        print(f'\tPrecision: {p}')
        print(f'\tPrecision@k: {p_k}')
    print('~'*100 + '\n')
    return recommendations, recommendations_attr

## Tune feature weights

In [25]:
test_weights = [ (1, 0), (0.75, 0.25), 'equal', 'scale', (0.25, 0.75), (0, 1) ]
# query_users = np.random.choice(user_item_matrix.index, size=1, replace=False)
# query_users = ['ASD5WK2ST6GAU']
# query_users = ['A3D7CZZF99WAQ6']
query_users = ['A29CEI6U2BLLCK']

# try all weights
for wts in test_weights:
    print('='*100)
    print(f'FEATURE WEIGHTS (ratings, categories): {wts}')
    print('='*100)
    similarity_matrix, dist_matrix = weighted_sim(feats=(rating_similarity, cat_similarity), weights=wts)
    knn = NearestNeighbors(metric='precomputed')
    knn.fit(dist_matrix.values)
    # for each query user
    for query_user in query_users:
        query(query_user, knn, show_eval=True)
    print('\n\n\n\n')

FEATURE WEIGHTS (ratings, categories): (1, 0)
Querying user: A29CEI6U2BLLCK

Positive reviews: 
asin
B000ZKBJXM    5.0
B001PPGG2K    5.0
B005FYJA52    5.0
B00D6NUS14    5.0
Name: A29CEI6U2BLLCK, dtype: float64

Reference titles:
  - Ghostbusters - Nintendo DS (item B000ZKBJXM)
  - Nintendo DSi Car Adapter (item B001PPGG2K)
  - Super Mario Galaxy (Nintendo Selects) (item B005FYJA52)
  - Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Skylanders SWAP FORCE LIGHTCORE Figure Enchanted Star Strike (item B00IZPXGIU)
    -- Based on Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)
       Similarity: 0.24902912254587614
  2) Skylanders SWAP Force: Lightcore Grim Creeper Character (item B00F3I2UFC)
    -- Based on Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)
       Similarity: 0.24272805933797104
  3) Skylanders SWAP Force Fie


Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Skylanders SWAP FORCE LIGHTCORE Figure Enchanted Star Strike (item B00IZPXGIU)
    -- Based on Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)
       Similarity: 0.24775313490543377
  2) Skylanders SWAP Force: Lightcore Grim Creeper Character (item B00F3I2UFC)
    -- Based on Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)
       Similarity: 0.24160081374616357
  3) Skylanders SWAP Force Fiery Forge Battle Pack (item B00E5M5LFS)
    -- Based on Skylanders Giants: Lightcore Hex Character (item B00D6NUS14)
       Similarity: 0.22430457508765977
  4) Nintendo DS Starter Kit (item B00069BUXG)
    -- Based on Nintendo DSi Car Adapter (item B001PPGG2K)
       Similarity: 0.21819018776177573
  5) Nicktoons MLB - Nintendo DS (item B0050GGX28)
    -- Based on Ghostbusters - Nintendo DS (item B000ZKBJXM)
       Similarity: 0.21214755070403757

## Find recommendations based on user reviews

In [26]:
best_wts = (0.75, 0.25)
query_users = np.random.choice(user_item_matrix.index, size=25, replace=False)

print('='*100)
print(f'FEATURE WEIGHTS (ratings, categories): {best_wts}')
print('='*100)
similarity_matrix, dist_matrix = weighted_sim(feats=(rating_similarity, cat_similarity), weights=best_wts)
knn = NearestNeighbors(metric='precomputed')
knn.fit(dist_matrix.values)
# for each query user
for query_user in query_users:
    query(query_user, knn, show_eval=True)

FEATURE WEIGHTS (ratings, categories): (0.75, 0.25)
Querying user: A3V5TRUA8GA4IN

Positive reviews: 
asin
B000WMEEC6    4.0
B00HWMP0OU    5.0
B019H4DSI0    5.0
Name: A3V5TRUA8GA4IN, dtype: float64

Reference titles:
  - Sid Meier's Civilization Revolution - Nintendo DS (item B000WMEEC6)
  - Disney Magical World - Nintendo 3DS (item B00HWMP0OU)
  - GameSir G3w USB Wired PC Game Controller Dual Shock Joystick Gamepad for PC Windows 7/8/8.1/10 &amp; Android Smartphone/Tablet/TV BOX &amp; PS3 (item B019H4DSI0)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Adesso PCK-308W Wired Keyboard (item B000XJNTMY)
    -- Based on Sid Meier's Civilization Revolution - Nintendo DS (item B000WMEEC6)
       Similarity: 0.30504003478100755
  2)  Hoyle Casino 2009 [Old Version] (item B001B5HBCI)
    -- Based on Sid Meier's Civilization Revolution - Nintendo DS (item B000WMEEC6)
       Similarity: 0.2978020631107473
  3) Ch

  - Shadow OPS Red Mercury - Xbox (item B0000V48KM)
  - Star Wars Battlefront - Xbox (item B0001DI6FM)
  - Mech Assault 2 Lone Wolf - Xbox (item B0006BK5AI)
  - Star Wars Episode III Revenge of the Sith - Xbox (item B0007SL202)
  - Medal of Honor European Assault - Xbox (item B00095LEEW)
  - Destroy All Humans - Xbox (item B00097IA88)
  - Star Wars Battlefront II - Xbox (item B0009O7HUI)
  - Tom Clancy's Ghost Recon Advanced Warfighter - Xbox 360 (item B000A0EFJW)
  - Tom Clancy's Ghost Recon Advanced Warfighter 2 - Xbox 360 (item B000LXIO6I)
  - Ace Combat 6: Fires of Liberation (Platinum Hits) (item B000P297JS)
  - Ace Combat Assault Horizon - Xbox 360 (item B0042GW7C2)
  - Stronghold 3 Gold Edition - PC (item B006ZPAYJ6)
  - 120GB 120G Internal HDD Hard Drive Disk Disc for Xbox360 XBOX 360 S Slim Games (item B014SIVGAW)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Dai Senryaku VII: Modern Military T


Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Time Crisis: Razing Storm - Playstation 3 (item B003S55EWI)
    -- Based on House of the Dead OVERKILL - Extended Cut - Playstation 3 (item B0053B7ICO)
       Similarity: 0.3175747066700974
  2) Resident Evil: Operation Raccoon City - Playstation 3 (item B004UDB9SA)
    -- Based on House of the Dead OVERKILL - Extended Cut - Playstation 3 (item B0053B7ICO)
       Similarity: 0.3095988414136764
  3) Shadows of the Damned - Playstation 3 (item B00432O50W)
    -- Based on House of the Dead OVERKILL - Extended Cut - Playstation 3 (item B0053B7ICO)
       Similarity: 0.3032476126369057
  4) F-Zero GP Legend (item B00022GIYI)
    -- Based on House of the Dead OVERKILL - Extended Cut - Playstation 3 (item B0053B7ICO)
       Similarity: 0.3028447541916386
  5) Men of Valor - Xbox (item B0001R0426)
    -- Based on House of the Dead OVERKILL - Extended Cut - Playstati

  - Halo 2 - Xbox (item B00008J7NZ)
  - Xbox Controller S-Blue (item B0000C6EDL)
  - Mortal Kombat Deception - Xbox (item B00019AZAK)
  - Fatal Frame 2 - Xbox (item B0002A6CQY)
  - NBA Live 2005 - Xbox (item B0002ILSJM)
  - Fable with Bonus DVD (item B0002IZICK)
  - Sonic Mega Collection Plus - Xbox (item B0002VYPEE)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Garfield's FunFest (item B000W08A0A)
    -- Based on Fatal Frame 2 - Xbox (item B0002A6CQY)
       Similarity: 0.4145750235203326
  2) NCAA Football 2006 - Xbox (item B00083G5CQ)
    -- Based on NBA Live 2005 - Xbox (item B0002ILSJM)
       Similarity: 0.407285941869007
  3) Sonic The Hedgehog 4 Episode 2 [Download] (item B007XZ4A7E)
    -- Based on Sonic Mega Collection Plus - Xbox (item B0002VYPEE)
       Similarity: 0.3949568901432422
  4) Teenage Zombies - Nintendo DS (item B00025LDLI)
    -- Based on Fatal Frame 2 - Xbox (item B0002A6CQY)
 

    -- Based on God of War - PlayStation 2 (item B0002XL3BA)
       Similarity: 0.2958902293818226
  6) Mark of Kri - PlayStation 2 (item B000066JRQ)
    -- Based on God of War - PlayStation 2 (item B0002XL3BA)
       Similarity: 0.2954198342165305
  7) Freedom Fighters The Battle for Liberty Island - PlayStation 2 (item B000088KH5)
    -- Based on God of War - PlayStation 2 (item B0002XL3BA)
       Similarity: 0.29453194647477704
  8) The Simpsons Hit &amp; Run (item B000095ZH5)
    -- Based on God of War - PlayStation 2 (item B0002XL3BA)
       Similarity: 0.2933555764178367
  9) Jak and Daxter: The Precursor Legacy (item B00005R5PO)
    -- Based on God of War - PlayStation 2 (item B0002XL3BA)
       Similarity: 0.2926563975997231
----------------------------------------------------------------------------------------------------
Recommendation evaluation:
	Hits: 0
	Recall: 0.0
	Precision: 0.0
	Precision@k: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  - Nintendo Nunchuk Controller - White (item B0094X2066)
  - Nintendo Wii Remote Plus - White (item B0094X227I)
  - Redragon M901 Gaming Mouse Wired, [Programmable] MMO RGB LED Mice, 24000 DPI, Laser High Precision Sensor, Weight Tuning Set, 18 Buttons for Windows PC Games - Black (item B00GU8W5AE)
  - Xbox One Wireless Controller [Without Bluetooth] (item B00YJJ0OQS)
  - XLon Gaming Headset Over Ear Headphone Stereo Surround Sound Earpiece Noise Cancelling Earphone with Mic for PS4, Xbox One, Nintendo Switch/3DS/New 3Dsii, Laptop, Computer, Tablet, iPad, Phones Etc (item B01C81OQEY)

Top 10 recommendations
----------------------------------------------------------------------------------------------------
  1) Nintendo Wii Remote Plus, Yoshi (item B00K73DP5W)
    -- Based on Nintendo Wii Remote Plus - White (item B0094X227I)
       Similarity: 0.318853539442618
  2) Wii U Fit Meter (item B00FF9MLQW)
    -- Based on Nintendo Wii Remote Plus - White (item B0094X227I)
       Similarity:

  6) Assassin's Creed: Revelations (item B004YVOCV4)
    -- Based on Assassin's Creed IV Black Flag - Playstation 3 (item B00BMFIXKQ)
       Similarity: 0.2931886198432021
  7) Metal Gear Solid HD Collection (item B0050SX97I)
    -- Based on Mortal Kombat: Komplete Edition - Playstation 3 (item B006ZTHFPS)
       Similarity: 0.29241270318371565
  8)  WWE SmackDown vs. Raw 2009 - PlayStation 2 (item B00175GC4K)
    -- Based on Mortal Kombat: Komplete Edition - Playstation 3 (item B006ZTHFPS)
       Similarity: 0.2923759383662183
  9) PS3 500 GB Grand Theft Auto V Bundle (item B00DBKSN8M)
    -- Based on Mortal Kombat: Komplete Edition - Playstation 3 (item B006ZTHFPS)
       Similarity: 0.2919123385951135
  10) Need for Speed: The Run - Playstation 3 (item B003O6ECOM)
    -- Based on Mortal Kombat: Komplete Edition - Playstation 3 (item B006ZTHFPS)
       Similarity: 0.29070897480432184
----------------------------------------------------------------------------------------------------


    -- Based on Neverwinter Nights - PC (item B00004TSXC)
       Similarity: 0.32908413134841846
  7) Command &amp; Conquer: Generals - PC (item B00007LVJD)
    -- Based on Neverwinter Nights - PC (item B00004TSXC)
       Similarity: 0.3275552518699757
  8) The Sims 3: Pets Expansion Pack (item B0054IV0A2)
    -- Based on The Sims 3: Generations - Expansion Pack PC/Mac (item B004UJNN5G)
       Similarity: 0.32593769914537124
  9) Madden NFL 08 - Nintendo DS (item B000P0SEPS)
    -- Based on Fight Night Round 4 - Xbox 360 (item B001989B4S)
       Similarity: 0.3256910345830375
  10) Gradius III And IV -  PlayStation 2 (item B00004U62O)
    -- Based on Twisted Metal: Black (item B00004TN8R)
       Similarity: 0.32107322335253174
----------------------------------------------------------------------------------------------------
Recommendation evaluation:
	Hits: 1
	Recall: 0.05555555555555555
	Precision: 0.00625
	Precision@k: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    -- Based on Shantae: Half-Genie Hero - Risky Beats Edition - PlayStation 4 (item B01GTVDX16)
       Similarity: 0.3052907335580864
  7) Aliens Infestation (item B005C2QSDY)
    -- Based on Yomawari: Night Alone / htol#NiQ: The Firefly Diary - PlayStation Vita Limited Edition (item B01FWKKVTM)
       Similarity: 0.3045154891738057
  8) Putty Squad (item B00HYPWEBW)
    -- Based on Shantae: Half-Genie Hero - Risky Beats Edition - PlayStation 4 (item B01GTVDX16)
       Similarity: 0.3025812482193736
  9) DuckTales: Remastered [Online Game Code] (item B00E0S4ZM2)
    -- Based on Shantae: Half-Genie Hero - Risky Beats Edition - PlayStation 4 (item B01GTVDX16)
       Similarity: 0.2993982404233343
  10) FINAL FANTASY XIV: Heavensward - Windows (item B00OM6SAK0)
    -- Based on Final Fantasy XIV Online: 60 Day Time Card [Online Game Code] (item B00O15486W)
       Similarity: 0.2984044873795191
------------------------------------------------------------------------------------------------